In [69]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import pandas as pd
import plotly.plotly as py

Description: the below code evaluates all the models using MAP and NDCG@10 metrics and compares the models based on these metrics.

Data Upload:

In [25]:
#Loading the test file:
def load_test_file(file_test):
    #original test.txt (Fold 1) with relevance values:
    df_test = pd.read_csv(file_test, sep=" ", header = None)
    #data cleaning:
    df_test[1] = df_test[1].map(lambda x: x.lstrip('qid:'))      
    df_test.insert(0, 'quid', df_test[1])
    df_test.insert(1, 'rel', df_test[0])
    df_test = pd.concat([df_test['quid'], df_test['rel']], axis=1, keys=['quid', 'rel'])
    return df_test

#Loading the reranked file with the scores: 
def load_reranked_file(file_reranked):
    model = pd.read_csv(file_reranked, sep="	", header = None)
    model.columns = ['quid_model','count_model', 'rank_model']
    return model

#Concatinates files: includes original relevance labels from the test.txt file and sorts the scores in descending order to 
#in order to provide final ranking by the model:
def final_ranking(df_test, model):
    df = pd.concat([df_test['quid'], df_test['rel'], model['rank_model']], axis=1)
    df_new_rank = df.groupby('quid').apply(lambda x: x.sort_values(['rank_model'],ascending = False))
    df_new_rank['rank'] = df_new_rank.groupby('quid').cumcount()+1
    return df_new_rank

MAP:

In [26]:
def MAP(df_new_rank):
    df_MAP = pd.concat([df_new_rank['quid'], df_new_rank['rel'], df_new_rank['rank_model'],df_new_rank['rank']], axis=1)
    
    #expressing relevance in binary format
    df_MAP['rel'] = np.where(df_MAP['rel'] >=1, 1,0)
    
    #inserting column with cumulative sum of relevance:
    df_MAP['rel_cum_sum'] = df_MAP.groupby('quid')['rel'].cumsum()
    
    # Precision:
    df_MAP['precision'] = (df_MAP['rel_cum_sum']*df_MAP['rel'])/df_MAP['rank']
    
    # AP - average precision per query:
    precision_sum = df_MAP.groupby('quid')['precision'].sum()
    rel_sum = df_MAP.groupby('quid')['rel'].sum()
    AP = precision_sum / rel_sum
    
    # MAP - mean average precision of the system for all queries
    MAP = AP.sum() / len(AP)
    return MAP

NDCG@10:

In [27]:
#Discounted Cumulative Gain (DCG):

def NDCG10(df_new_rank):
    #Discounted Cumulative Gain (DCG):
    #creating a new dataframe with quid, rel and rank for DCG Dicounted Cumulative Gain
    df_NDCG = pd.concat([df_new_rank['quid'], df_new_rank['rel'], df_new_rank['rank']], axis=1)
    df_NDCG = df_NDCG.groupby('quid').head(10)
    #Gain:
    df_NDCG['gain'] = (2**df_NDCG['rel'])-1
    df_NDCG['rank+1'] = df_NDCG['rank']+1
    #Discount by Rank:
    df_NDCG['discount'] = np.log2(df_NDCG['rank+1'])
    #Discounted Gain:
    df_NDCG['discounted_gain'] = df_NDCG['gain']/df_NDCG['discount']
    #DCG - Discounted Cumulative Gain per quid:
    DCG = df_NDCG.groupby(by=['quid'])['discounted_gain'].sum()
    
    #Optimal Discounted Cumulitave Gain (optDCG):
    #optDCG: optimal Dicounted Cumulative Gain
    df_opt_NDCG = pd.concat([df_new_rank['quid'], df_new_rank['rel']], axis=1, keys=['quid', 'rel'])
    opt_NDCG = df_opt_NDCG.groupby('quid')['rel'].apply(lambda x: x.order(ascending=False)).reset_index()
    opt_NDCG['rank'] = opt_NDCG.groupby('quid').cumcount()+1
    opt_NDCG = opt_NDCG.groupby('quid').head(10)
    #opt DCG: Dicounted Cumulative Gain
    #Gain:
    opt_NDCG['gain'] = (2**opt_NDCG['rel'])-1
    opt_NDCG['rank+1'] = opt_NDCG['rank']+1
    #Discount by Rank:
    opt_NDCG['discount'] = np.log2(opt_NDCG['rank+1'])
    #Discounted Gain:
    opt_NDCG['discounted_gain'] = opt_NDCG['gain']/opt_NDCG['discount']
    #opt DCG - Discounted Cumulative Gain per quid:
    optDCG = opt_NDCG.groupby(by=['quid'])['discounted_gain'].sum()

    #Normalized Discounted Cumulative Gain (NDCG):
    #avg NDCG for the system:
    NDCG = DCG/optDCG
    NDCG = np.nan_to_num(NDCG)
    avg_NDCG = sum(NDCG)/len(NDCG)
    return avg_NDCG


In [44]:
#Defining test.txt file and all 13 models:
file_test = 'test.txt'
file_reranked_1= 'RankNetRerank/reranked_trial1.txt'
file_reranked_2= 'RankNetRerank/reranked_trial2.txt'
file_reranked_3= 'RankNetRerank/reranked_trial3.txt'
file_reranked_4= 'RankNetRerank/reranked_trial4.txt'
file_reranked_5= 'RankNetRerank/reranked_trial5.txt'
file_reranked_6= 'RankNetRerank/reranked_trial6.txt'
file_reranked_7= 'RankNetRerank/reranked_trial7.txt'
file_reranked_8= 'RankNetRerank/reranked_trial8.txt'
file_reranked_9= 'RankNetRerank/reranked_trial9.txt'
file_reranked_10= 'RankNetRerank/reranked_trial10.txt'
file_reranked_11= 'RankNetRerank/reranked_trial11.txt'
file_reranked_12= 'RankNetRerank/reranked_trial12.txt'
file_reranked_13= 'RankNetRerank/reranked_trial13.txt'


#Loading test.txt file and all 13 models:
test = load_test_file(file_test)
model1 = load_reranked_file(file_reranked_1)
model2 = load_reranked_file(file_reranked_2)
model3 = load_reranked_file(file_reranked_3)
model4 = load_reranked_file(file_reranked_4)
model5 = load_reranked_file(file_reranked_5)
model6 = load_reranked_file(file_reranked_6)
model7 = load_reranked_file(file_reranked_7)
model8 = load_reranked_file(file_reranked_8)
model9 = load_reranked_file(file_reranked_9)
model10 = load_reranked_file(file_reranked_10)
model11 = load_reranked_file(file_reranked_11)
model12 = load_reranked_file(file_reranked_12)
model13 = load_reranked_file(file_reranked_13)


In [57]:
#Evaluating all models:
df1 = final_ranking(test, model1)
df2 = final_ranking(test, model2)
df3 = final_ranking(test, model3)
df4 = final_ranking(test, model4)
df5 = final_ranking(test, model5)
df6 = final_ranking(test, model6)
df7 = final_ranking(test, model7)
df8 = final_ranking(test, model8)
df9 = final_ranking(test, model9)
df10 = final_ranking(test, model10)
df11 = final_ranking(test, model11)
df12 = final_ranking(test, model12)
df13 = final_ranking(test, model13)


In [61]:
MAP_val = [MAP(df1), MAP(df2), MAP(df3),MAP(df4),MAP(df5),MAP(df6),MAP(df7),MAP(df8),MAP(df9),MAP(df10),MAP(df11),MAP(df12),MAP(df13)]
NDCG10_val = [NDCG10(df1), NDCG10(df2), NDCG10(df3),NDCG10(df4),NDCG10(df5),NDCG10(df6),NDCG10(df7),NDCG10(df8),NDCG10(df9),NDCG10(df10),NDCG10(df11),NDCG10(df12),NDCG10(df13)]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:21: FutureWarning: order is deprecated, use sort_values(...)


In [95]:
models_all = ('Model1', 'Model2', 'Model3', 'Model4', 'Model5', 'Model6', 'Model7', 'Model8','Model9','Model10','Model11','Model12','Model13')
df = pd.DataFrame({'NDCG@10':NDCG10_val, 'Model': models_all,'MAP':MAP_val})
df = df[['Model', 'MAP', 'NDCG@10']]
df

,Model,MAP,NDCG@10
0,Model1,0.470518,0.197097
1,Model2,0.475446,0.199574
2,Model3,0.460084,0.192611
3,Model4,0.475702,0.197435
4,Model5,0.471130,0.180065
5,Model6,0.465515,0.180614
6,Model7,0.466614,0.183121
7,Model8,0.468796,0.196259
8,Model9,0.467075,0.183019
9,Model10,0.468495,0.193129
